In [ ]:
import csv
import pandas
import numpy
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
import cv2

In [ ]:
!mkdir log
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

LOG_DIR = './log/' 
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
def show_all(arg):
    import math
    plt.figure(figsize=(10,10))
    index = 1
    myLen = len(arg)
    row = math.ceil(math.sqrt(myLen))
    myCol = myLen // row
    for i in range(myCol + 1):
        for j in range(row):
            #print(myCol + 1, row)
            if i * row + j < myLen :
                t_img = arg[i * row + j]
                plt.subplot(myCol + 1, row, i * row + j + 1)
                plt.imshow(t_img)
                plt.title(index)
                index += 1
                plt.xticks([]), plt.yticks([])
            else:
                plt.show()
                return

In [ ]:
def get_generator(path, classes, size):
    image_data_generator=ImageDataGenerator(
        rescale=1./255.)
    training_generator = image_data_generator.flow_from_directory(
        classes = classes,
        directory = path,
        batch_size = 1,
        seed = 42,
        shuffle = True,
        target_size = size)
    return training_generator
painting_generator = get_generator('../input/art-images-drawings-painting-sculpture-engraving/dataset',None, (100, 100))
indoor_generator = get_generator('../input/indoor-training-set-its-residestandard', ['clear'],(200, 200))

In [ ]:
pic_generator = get_generator('../input/painting', ['╤╣╦⌡'],(200, 200))

In [ ]:
def load_back():
    back = next(indoor_generator)[0].reshape((200, 200, 3))
    back = cv2.cvtColor(back, cv2.COLOR_RGB2RGBA)
    return back
def load_front():
    front = next(painting_generator)[0].reshape((100, 100, 3))
    front = cv2.cvtColor(front, cv2.COLOR_RGB2RGBA)
    transparent_layer = numpy.zeros((200, 200, 4))
    transparent_layer[50:150, 50:150] = front
    transparent_layer = transparent_layer.astype(numpy.float32)
    return transparent_layer
def transform(front):
    from random import uniform
    raw_points = numpy.float32([ [50,50], [50,149], [149,149], [149,50] ])
    new_points = numpy.float32([ 
        [uniform(10, 60), uniform(10, 60)], 
        [uniform(10, 60), uniform(140, 190)], 
        [uniform(140, 190), uniform(140, 190)],  
        [uniform(140, 190), uniform(10, 60)]
    ])
    M = cv2.getPerspectiveTransform(raw_points, new_points)
    front = cv2.warpPerspective(front, M, (200, 200), flags=cv2.INTER_NEAREST)
    # front = cv2.GaussianBlur(front,(3,3),0)
    return (front, new_points.reshape((8,)))
def merge(back, front):
    ret, mask = cv2.threshold(cv2.split(front)[-1],0.5,1,cv2.THRESH_BINARY_INV)
    mask = mask.astype(numpy.uint8)
    back = cv2.bitwise_and(back,back,mask = mask)
    del mask
    img = cv2.add(back, front)
    return cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
def canny(mat, ksize):
    gray = cv2.cvtColor(mat,cv2.COLOR_RGB2GRAY)
    gauss = cv2.GaussianBlur(gray, (ksize, ksize), 0.3*((ksize-1)*0.5-1)+0.8)
    edged = cv2.Canny(gauss, 50, 60)
    return edged
def process_img(img):
#     raw = img.copy() 
#     ksize = 7
# #     epsilon = 3
#     img = img*255
#     img = img.astype(numpy.uint8)
#     img = canny(img, ksize)
#     lines = cv2.HoughLinesP(img, 1, numpy.pi / 360, 30,minLineLength=30,maxLineGap=6)
# #     print(lines)
#     for line in lines:
#         x1, y1, x2, y2 = line[0]
#         cv2.line(raw, (x1, y1), (x2, y2), (255, 255, 0), 2)
#     cnts = Countours(img)
#     img = approxPoly(img, cnts, epsilon)
#     mask = draw_mask(img)
#     img = cv2.GaussianBlur(img,(3,3),0)*255
#     img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY).astype(numpy.uint8)
#     img = cv2.Canny(img, 50, 150)
#     img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)/255
#     img = img.astype(numpy.float32)
#     img = cv2.add(img, raw)
    return img
def get_an_img():
    back = load_back()
    front = load_front()
    front, points = transform(front)
    img = merge(back, front)
    del back
    del front
    img = process_img(img)
    return (img, points)
show_all([get_an_img()[0]])

In [ ]:
import sys
# sys.getsizeof([get_an_img() for i in range(100)])
test_x = []
test_y = []
for i in range(100):
    try:
        img, points = get_an_img()
        test_x.append(img)
        test_y.append(points/200)
#         print(i)
    except:
        print(i, "XXXXXXXXXXXXXXXXXXXXXXXXXXXX")

In [ ]:
import sys
# sys.getsizeof([get_an_img() for i in range(100)])
x = []
y = []
for i in range(4000):
    try:
        img, points = get_an_img()
        x.append(img)
        y.append(points/200)
#         print(i)
    except:
        print(i, "XXXXXXXXXXXXXXXXXXXXXXXXXXXX")

In [ ]:
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(200, 200, 3))
# conv_base.trainable = False
conv_base.trainable = True 
# set_trainable = False 
# for layer in conv_base.layers:     
#     if layer.name == 'block5_conv1':         
#         set_trainable = True     
#     if set_trainable:         
#         layer.trainable = True     
#     else:         
#         layer.trainable = False
model = models.Sequential()
model.add(conv_base)
# model.add(layers.Conv2D(32, (3, 3), activation='relu',
#                         input_shape=(100, 100, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(256, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
# model.add(layers.Dense(2048, activation='relu'))
# model.add(layers.Dropout(0.4))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.4))
layers.BatchNormalization()
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.4))
layers.BatchNormalization()
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(8, activation='sigmoid'))
model.compile(
    loss='mae',
    optimizer=optimizers.RMSprop(lr=1e-5),
    metrics=['mae'])

In [ ]:
from keras.callbacks import Callback
import random
def draw_points(img, points):
    import copy
    point_size = 5
    point_color = (0,255,0)
    thickness = -1
    points = points*200
    image = copy.deepcopy(img)
    cv2.circle(image, (points[0], points[1]), point_size, point_color, thickness)
    cv2.circle(image, (points[2], points[3]), point_size, point_color, thickness)
    cv2.circle(image, (points[4], points[5]), point_size, point_color, thickness)
    cv2.circle(image, (points[6], points[7]), point_size, point_color, thickness)
    return image
class Show_effect(Callback):
    def __init__(self):
        super(Show_effect, self).__init__()
        self.epoch = -1
    def on_epoch_end(self, batch, logs={}):
        self.epoch = self.epoch + 1
        if(self.epoch % 8 ==0):
            data_for_test = [random.choice(test_x) for i in range(4)]
            for i in range(4):
                data_for_test.append(process_img(next(pic_generator)[0].reshape((200,200,3))))
            show_all([draw_points(img,model.predict(numpy.expand_dims(img,axis=0)).reshape((8,))) for img in data_for_test])
show_effect = Show_effect()

In [ ]:
x=numpy.array(x)
y=numpy.array(y)
model.fit(
    x,
    y,
    batch_size=64,
    epochs=1000,
    validation_split=0.3,
    callbacks=[TensorBoard(log_dir='./log'),show_effect])

In [ ]:
data_for_test = [random.choice(test_x)  for i in range(4)]
for i in range(4):                     
    data_for_test.append(process_img(next(pic_generator)[0].reshape((200,200,3))))
show_all([draw_points(img,model.predict(numpy.expand_dims(img,axis=0)).reshape((8,))) for img in data_for_test])

In [ ]:
x=numpy.array(x)
p=model.predict(x[:1]) 
t=y[:1]
print(p-t)
print(sum((p-t)*(p-t))) 
print(p)
print(t) 

In [ ]:
model.save("image.h5") 